# VNA V3

Want to be able to live plot both S-parameters and the data-read in at once. Also cleans up the imports.

In [1]:
import sys
import os
import importlib #
sys.path.append(os.path.abspath('../scripts'))

#widgets and display
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Output
import pprint

#Plotting 
from bqplot import pyplot as plt
import threading
from threading import Thread

#caculations
import numpy as np 
from scipy import signal
import time
import math

# Import functions from custom scripts
from sig_source import SigSource
from sig_source_ui import SignalSourceUI
from data_proccess import VNAfunc

In [ ]:
#Import Pynq
from pynq import PL
from pynq import allocate
from pynq import Overlay
import xrfdc

### Upload Code to RFSOC

In [ ]:
rfsoc_button = widgets.Button(description="Print RFSOC Code Lines")

rfosc_out = Output()
def print_rfsoc(func):
    with rfosc_out:
        rfosc_out.clear_output
        try: 
            pprint.pprint(ol.ip_dict)
        except Exception as e:
            print(f"Error: {e}")


PL.reset() #important fixes caching issues which have popped up.
ol = Overlay('./design_1.bit')  #locate/point to the bit file
dma_interface = ol.axi_dma_0
rf = ol.usp_rf_data_converter_0

rfsoc_button.on_click(print_rfsoc)
display(widgets.VBox([widgets.Label(value="Print RFSOC Output"), rfsoc_button, rfosc_out])) #Display button only after the code is uploaded 

### Signal Source Generation 

In [2]:
# Create and display the UI for signal generation
sig_ui = SignalSourceUI()
sig_ui.display()

### Real Time Output Plot

In [ ]:
fs = 147.456e6 # Sampling frequency
n = 65536 # Number of samples
T = n/fs

In [ ]:
#Import Relavent Functions 
def read_dma():
    # Trigger the DMA transfer and wait for the result
    out_buffer = allocate(400024 * 4, dtype=np.int32)
    # Trigger the DMA transfer and wait for the result
    dma_interface.recvchannel.transfer(out_buffer)
    dma_interface.recvchannel.wait()
    return out_buffer

In [ ]:
# Global control variable
running = False

# Widgets with layout adjustments
mode_toggle = widgets.ToggleButtons(
    options=['User Input Frequency', 'Generate Points'],
    description='Mode:',
    disabled=False,
    button_style='',
    layout=widgets.Layout(
        width='auto',         # Automatically adjust width based on content
        min_width='400px',    # Minimum width to prevent truncation
        max_width='1000px'    # Max width to avoid excessive stretching
    )
)
user_freq_input = widgets.FloatText(
    description='Freq (Hz):',
    value=10e6,  # Example starting frequency
    step=1,
    layout=widgets.Layout(width='300px')  # Adjust width to fit description
)
set_freq_button = widgets.Button(
    description="Set Frequency",
    layout=widgets.Layout(width='150px')  # Adjust width for clarity
)
generate_freq_button = widgets.Button(
    description="Generate Frequency Points",
    layout=widgets.Layout(width='200px')  # Adjust width to fit text
)
# Output widget
output = widgets.Output(layout=widgets.Layout(width='500px'))  # Adjust width for output clarity

# Functions
def freq_update_loop(frequency_list):
    global running
    for freq in frequency_list:
        if not running:  # Check if the loop should be stopped
            break
        # Simulate frequency update (replace this with the actual signal generation function)
        with output:
            clear_output(wait=True)
            print(f"Setting freq to {freq:.2f}")
            sig_ui.source.set_frequency()
        time.sleep(0.3)  # Simulate delay between frequency updates

def set_user_frequency(change=None):
    with output:
        clear_output()
        try:
            if mode_toggle.value == "User Input Frequency":
                frequency = user_freq_input.value
                # Replace this with actual method to set the frequency in your signal source
                print(f"Set frequency to {frequency} Hz.")
            else:
                print(f"Wrong Mode. Following automatic Frequency")
        except Exception as e:
            print(f"Error setting frequency: {e}")

def generate_freq_points(change=None):
    with output:
        clear_output()
        try:
            if mode_toggle.value == "Generate Points":
                frequency_list = sig_ui.source.generate_freq_points()
                # Start the frequency update loop in a new thread
                thread = threading.Thread(target=freq_update_loop, args=(frequency_list,), daemon=True)
                thread.start()
            else:
                print(f"Wrong Mode. Following user input Frequency")
        except Exception as e:
            print(f"Error generating frequency points: {e}")

# Handle mode changes to stop the loop
def on_mode_toggle_change(change):
    global running
    if mode_toggle.value == "User Input Frequency":
        running = False
        with output:
            clear_output(wait=True)
            print("Mode changed to User Input Frequency. Stopping frequency generation.")
    else:
        running = True
        with output:
            clear_output(wait=True)
            print("Mode changed to Generate Frequency Points. Hit Generate Frequency Points to start sweep.")

# Widgets event bindings
set_freq_button.on_click(set_user_frequency)
generate_freq_button.on_click(generate_freq_points)
mode_toggle.observe(on_mode_toggle_change, names='value')

# Layout
layout = widgets.VBox([
    mode_toggle,
    widgets.HBox([user_freq_input, set_freq_button]),
    generate_freq_button,
    output
])
display(layout)


Setting freq to 10000000.00


Setting freq to 211919191.92


Setting freq to 413838383.84


Setting freq to 615757575.76


Setting freq to 817676767.68


...

Setting freq to 1423434343.43


Setting freq to 615757575.76


Setting freq to 1625353535.35


Setting freq to 817676767.68


Setting freq to 1827272727.27


In [41]:
import numpy as np
import bqplot as bq
import ipywidgets as widgets
from ipywidgets import Layout
import time
import threading

# Initial data for the plots
x = np.linspace(0, 10, 100)
freq_sine = 1
freq_cosine = 1

# Create the first scatter plot (Sine)
scatter1 = bq.Scatter(x=x, y=np.sin(freq_sine * x), 
                      scales={'x': bq.LinearScale(), 'y': bq.LinearScale()},
                      labels=['sin(x)'])
ax_scatter1 = bq.Axis(scale=scatter1.scales['x'], label='X-axis')
ay_scatter1 = bq.Axis(scale=scatter1.scales['y'], orientation='vertical', label='Y-axis')
fig1 = bq.Figure(marks=[scatter1], axes=[ax_scatter1, ay_scatter1], title="Sine Function")

# Create the second scatter plot (Cosine)
scatter2 = bq.Scatter(x=x, y=np.cos(freq_cosine * x), 
                      scales={'x': bq.LinearScale(), 'y': bq.LinearScale()},
                      labels=['cos(x)'])
ax_scatter2 = bq.Axis(scale=scatter2.scales['x'], label='X-axis')
ay_scatter2 = bq.Axis(scale=scatter2.scales['y'], orientation='vertical', label='Y-axis')
fig2 = bq.Figure(marks=[scatter2], axes=[ax_scatter2, ay_scatter2], title="Cosine Function")

# Create sliders to control the frequency
sine_freq_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Sine Freq:')
cosine_freq_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Cosine Freq:')

# Output widget for messages
output = widgets.Output()

# Function to update both sine and cosine plots
def update_plots(change=None):
    global freq_sine, freq_cosine
    
    # Get the current slider values
    freq_sine = sine_freq_slider.value
    freq_cosine = cosine_freq_slider.value
    
    # Update the data for both scatter plots
    scatter1.y = np.sin(freq_sine * x)
    scatter2.y = np.cos(freq_cosine * x)
    
    with output:
        clear_output(wait=True)
        print(f"Sine frequency: {freq_sine}, Cosine frequency: {freq_cosine}")

# Link slider events to the update function
sine_freq_slider.observe(update_plots, names='value')
cosine_freq_slider.observe(update_plots, names='value')

# Arrange the widgets and plots in a layout
layout = widgets.VBox([
    widgets.HBox([sine_freq_slider, cosine_freq_slider]),
    widgets.HBox([fig1, fig2]),
    output
])

# Display the layout
display(layout)

# Initial plot update
update_plots()
